It runs and moves in order from the top.
Settings include model changes.
>上から順に実行して動きます。
>設定はモデル変更などがあります。

Adapted from（参考元）: 

https://colab.research.google.com/github/camenduru/stable-diffusion-webui-colab/blob/main/stable-diffusion-v2-webui-colab.ipynb

In [ ]:
#@title Clone webUI repository, etc.（webUIリポジトリのクローンなど）
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui --quiet
%cd /content/stable-diffusion-webui

!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui-aesthetic-gradients extensions/stable-diffusion-webui-aesthetic-gradients --quiet
!git clone https://github.com/thx-pw/stable-diffusion-webui-models_downloader extensions/stable-diffusion-webui-models_downloader --quiet

!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui-old-localizations extensions/stable-diffusion-webui-old-localizations --quiet
!git clone https://github.com/dtlnor/stable-diffusion-webui-localization-zh_CN extensions/stable-diffusion-webui-localization-zh_CN --quiet
!git clone https://github.com/benlisquare/stable-diffusion-webui-localization-zh_TW extensions/stable-diffusion-webui-localization-zh_TW --quiet
!git clone https://github.com/36DB/stable-diffusion-webui-localization-ko_KR extensions/stable-diffusion-webui-localization-ko_KR --quiet
!git clone https://github.com/innovaciones/stable-diffusion-webui-localization-es_ES extensions/stable-diffusion-webui-localization-es_ES --quiet
!git clone https://github.com/Harvester62/stable-diffusion-webui-localization-it_IT extensions/stable-diffusion-webui-localization-it_IT --quiet
!git clone https://github.com/Strothis/stable-diffusion-webui-de_DE extensions/stable-diffusion-webui-de_DE --quiet
!git clone https://github.com/Katsuyuki-Karasawa/stable-diffusion-webui-localization-ja_JP extensions/stable-diffusion-webui-localization-ja_JP --quiet
!git clone https://github.com/M-art-ucci/stable-diffusion-webui-localization-pt_BR extensions/stable-diffusion-webui-localization-pt_BR --quiet
!git clone https://github.com/camenduru/stable-diffusion-webui-localization-tr_TR extensions/stable-diffusion-webui-localization-tr_TR --quiet
!git clone https://github.com/Cyanz83/stable-diffusion-webui-localization-no_NO extensions/stable-diffusion-webui-localization-no_NO --quiet

!pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.15/xformers-0.0.15.dev0+1515f77.d20221130-cp38-cp38-linux_x86_64.whl

In [ ]:
#@title Download the model from Hugging Face.（Hugging Faceからモデルをダウンロード）

models_dict = {
  "None" : "",
  "stable-v20-768" : "https://huggingface.co/stabilityai/stable-diffusion-2/blob/main/768-v-ema.ckpt",
  "stable-v20-512" : "https://huggingface.co/stabilityai/stable-diffusion-2-base/blob/main/512-base-ema.ckpt",
  "stable-v20-512-inpainting" : "https://huggingface.co/stabilityai/stable-diffusion-2-inpainting/blob/main/512-inpainting-ema.ckpt",
  "stable-v20-512-depth" : "https://huggingface.co/stabilityai/stable-diffusion-2-depth/blob/main/512-depth-ema.ckpt",
  "8528-v01" : "https://huggingface.co/852wa/8528-diffusion/blob/main/8528-diffusion-v0-1.ckpt",
  "trinart-derrida_final" : "https://huggingface.co/naclbit/trinart_derrida_characters_v2_stable_diffusion/blob/main/derrida_final.ckpt",
  "trinart-characters-19m" : "https://huggingface.co/naclbit/trinart_characters_19.2m_stable_diffusion_v1/blob/main/trinart_characters_it4_v1.ckpt",
  "waifu-v13-float16" : "https://huggingface.co/hakurei/waifu-diffusion-v1-3/blob/main/wd-v1-3-float16.ckpt",
  "robo-v1" : "https://huggingface.co/nousr/robo-diffusion/blob/main/models/robo-diffusion-v1.ckpt",
}

vaes_dict = {
  "None" : "",
  "trinart-derrida_final" : "https://huggingface.co/naclbit/trinart_derrida_characters_v2_stable_diffusion/blob/main/autoencoder_fix_kl-f8-trinart_characters.ckpt",
  "trinart-characters-19m" : "https://huggingface.co/naclbit/trinart_characters_19.2m_stable_diffusion_v1/blob/main/autoencoder_fix_kl-f8-trinart_characters.ckpt",
  "waifu-v14-anime2" : "https://huggingface.co/hakurei/waifu-diffusion-v1-4/blob/main/vae/kl-f8-anime2.ckpt",
  "sd-vae-ft-mse" : "https://huggingface.co/stabilityai/sd-vae-ft-mse-original/blob/main/vae-ft-mse-840000-ema-pruned.ckpt",
}

model = "trinart-derrida_final" #@param ['None', 'stable-v20-768', 'stable-v20-512', 'stable-v20-512-inpainting', 'stable-v20-512-depth', '8528-v01', 'trinart-derrida_final', 'trinart-characters-19m', 'waifu-v13-float16', 'robo-v1']
vae = "trinart-derrida_final" #@param ['None', 'trinart-derrida_final', 'trinart-characters-19m', 'waifu-v14-anime2', 'sd-vae-ft-mse']

def resolve_raw_url(url):
  url = url.replace("/blob/", "/resolve/")
  return url

if model != 'None':
  model_url = resolve_raw_url(models_dict[model])
  !wget {model_url} -O /content/stable-diffusion-webui/models/Stable-diffusion/{model}.ckpt

if model == 'stable-v20-768':
  !wget "https://github.com/Stability-AI/stablediffusion/raw/main/configs/stable-diffusion/v2-inference-v.yaml" -O /content/stable-diffusion-webui/models/Stable-diffusion/{model}.yaml
elif model == 'stable-v20-512':
  !wget "https://github.com/Stability-AI/stablediffusion/raw/main/configs/stable-diffusion/v2-inference.yaml" -O /content/stable-diffusion-webui/models/Stable-diffusion/{model}.yaml
elif model == 'stable-v20-512-inpainting':
  !wget "https://github.com/Stability-AI/stablediffusion/raw/main/configs/stable-diffusion/v2-inpainting-inference.yaml" -O /content/stable-diffusion-webui/models/Stable-diffusion/{model}.yaml
elif model == 'stable-v20-512-depth':
  !wget "https://github.com/Stability-AI/stablediffusion/raw/main/configs/stable-diffusion/v2-midas-inference.yaml" -O /content/stable-diffusion-webui/models/Stable-diffusion/{model}.yaml

if vae != 'None':
  vae_url = resolve_raw_url(vaes_dict[vae])
  !wget {vae_url} -O /content/stable-diffusion-webui/models/VAE/{vae}.vae.pt

!wget -q https://raw.githubusercontent.com/thx-pw/stable-diffusion-webui-colab/main/styles.csv -O /content/stable-diffusion-webui/styles.csv

localization = "ja_JP" #@param ['None', 'ar_AR', 'de_DE', 'es_ES', 'fr-FR', 'it_IT', 'ja_JP', 'ko_KR', 'no_NO', 'pt_BR', 'ru_RU', 'tr_TR', 'zh_CN', 'zh_TW']
!echo '{"localization":' > "/content/stable-diffusion-webui/config.json"
!echo '"'{localization}'"' >> "/content/stable-diffusion-webui/config.json"
!echo '}' >> "/content/stable-diffusion-webui/config.json"

In [ ]:
#@title Download the model from Google Drive.（Google Driveからモデルをダウンロード）

import os
import glob
from google.colab import drive # type: ignore

use_google_drive = False #@param {type:"boolean"}
sd_webui_gdrive = "/content/drive/MyDrive/sd/stable-diffusion-webui" #@param {type:"string"}
sd_webui_colab = "/content/stable-diffusion-webui"

create_directory_gdrive = False #@param {type:"boolean"}
#@markdown Find and copy the following files from the stable-diffusion-webui(sd_webui_gdrive) directory.
#@markdown > stable-diffusion-webui(sd_webui_gdrive)ディレクトリから以下のファイルを見つけてコピーします。

#@markdown + models/Stable-diffusion/\*.ckpt
#@markdown + models/Stable-diffusion/\*.yaml
#@markdown + models/VAE/\*.ckpt
#@markdown + models/VAE/\*.pt
#@markdown + models/Stable-diffusion/\*.vae.ckpt
#@markdown + models/Stable-diffusion/\*.vae.pt
#@markdown + embeddings/\*.pt
#@markdown + embeddings/\*.bin
#@markdown + models/hypernetworks/*.pt
#@markdown + extensions/aesthetic-gradients/aesthetic_embeddings/*.pt
#@markdown + styles.csv
#@markdown + config.json
#@markdown + ui-config.json
#@markdown + /content/drive/MyDrive/model.ckpt
#@markdown + /content/drive/MyDrive/AI/models/*.ckpt

model_checkpoints = "models/Stable-diffusion"
model_model_vaes = "models/Stable-diffusion"
model_vae_vaes = "models/VAE"
textual_inversion_embeddings = "embeddings"
hypernetworks = "models/hypernetworks"
aesthetic_embeddings = "extensions/aesthetic-gradients/aesthetic_embeddings"
config_list = ["styles.csv", "config.json", "ui-config.json"]

model_checkpoints_gdrive = os.path.join(sd_webui_gdrive, model_checkpoints)
model_model_vaes_gdrive = os.path.join(sd_webui_gdrive, model_model_vaes)
model_vae_vaes_gdrive = os.path.join(sd_webui_gdrive, model_vae_vaes)
textual_inversion_embeddings_gdrive = os.path.join(sd_webui_gdrive, textual_inversion_embeddings)
hypernetworks_gdrive = os.path.join(sd_webui_gdrive, hypernetworks)
aesthetic_embeddings_gdrive = os.path.join(sd_webui_gdrive, aesthetic_embeddings)

model_checkpoints_colab = os.path.join(sd_webui_colab, model_checkpoints)
model_model_vaes_colab = os.path.join(sd_webui_colab, model_model_vaes)
model_vae_vaes_colab = os.path.join(sd_webui_colab, model_vae_vaes)
textual_inversion_embeddings_colab = os.path.join(sd_webui_colab, textual_inversion_embeddings)
hypernetworks_colab = os.path.join(sd_webui_colab, hypernetworks)
aesthetic_embeddings_colab = os.path.join(sd_webui_colab, aesthetic_embeddings)

if use_google_drive:
  drive.mount("/content/drive")

  ckpt_path = "/content/drive/MyDrive/model.ckpt"
  if os.path.exists(ckpt_path):
    !cp {ckpt_path} {model_checkpoints_colab}
  for model_checkpoint in glob.glob(f"/content/drive/MyDrive/AI/models/*.ckpt"):
    !cp {model_checkpoint} {model_checkpoints_colab}
 
  for config in config_list:
    config_gdrive = os.path.join(sd_webui_gdrive, config)
    if os.path.exists(config_gdrive):
      !cp {config_gdrive} {sd_webui_colab}

  if create_directory_gdrive:
    os.makedirs(model_checkpoints_gdrive, exist_ok=True)
    os.makedirs(model_model_vaes_gdrive, exist_ok=True)
    os.makedirs(model_vae_vaes_gdrive, exist_ok=True)
    os.makedirs(textual_inversion_embeddings_gdrive, exist_ok=True)
    os.makedirs(hypernetworks_gdrive, exist_ok=True)
    os.makedirs(aesthetic_embeddings_gdrive, exist_ok=True)
  os.makedirs(hypernetworks_colab, exist_ok=True)

  files = glob.glob(f"{model_checkpoints_gdrive}/*.ckpt")
  files.extend(glob.glob(f"{model_checkpoints_gdrive}/*.yaml"))
  for model_checkpoint in files:
    !cp {model_checkpoint} {model_checkpoints_colab}

  files = glob.glob(f"{model_model_vaes_gdrive}/*.vae.ckpt")
  files.extend(glob.glob(f"{model_model_vaes_gdrive}/*.vae.pt"))
  for model_vae in files:
    !cp {model_vae} {model_model_vaes_colab}
    
  files = glob.glob(f"{model_vae_vaes_gdrive}/*.ckpt")
  files.extend(glob.glob(f"{model_vae_vaes_gdrive}/*.pt"))
  for model_vae in files:
    !cp {model_vae} {model_vae_vaes_colab}

  files = glob.glob(f"{textual_inversion_embeddings_gdrive}/*.pt")
  files.extend(glob.glob(f"{textual_inversion_embeddings_gdrive}/*.bin"))
  for textual_inversion_embedding in files:
    !cp {textual_inversion_embedding} {textual_inversion_embeddings_colab}

  for hypernetwork in glob.glob(f"{hypernetworks_gdrive}/*.pt"):
    !cp {hypernetwork} {hypernetworks_colab}
    
  for aesthetic_embedding in glob.glob(f"{aesthetic_embeddings_gdrive}/*.pt"):
    !cp {aesthetic_embedding} {aesthetic_embeddings_colab}
  
  drive.flush_and_unmount()

In [ ]:
#@title Launch the webUI.（webUIの起動）
%cd '/content/stable-diffusion-webui'

# RAM不足のため
import gc
from google.colab import drive
gc.collect()
drive.flush_and_unmount()

username = "user" #@param {type:"string"}
password = "pass" #@param {type:"string"}
tunnel = "gradio" #@param ["ngrok", "gradio"]
#@markdown get a token from for ngrok. https://dashboard.ngrok.com/get-started/your-authtoken （ngrokからトークンを取得）
ngrok_authtoken = "" #@param {type:"string"}
ngrok_region = "jp" #@param ["us", "eu", "au", "ap", "sa", "jp", "in"]


common_args = f"--force-enable-xformers --lowram --enable-console-prompts --enable-insecure-extension-access --disable-safe-unpickle --update-check --deepdanbooru --gradio-auth {username}:{password}"
if tunnel == "ngrok":
  !COMMANDLINE_ARGS="{common_args} --ngrok {ngrok_authtoken} --ngrok-region {ngrok_region}" python launch.py
else:
  !COMMANDLINE_ARGS="{common_args} --share" python launch.py


In [ ]:
#@title Save to zip file with same password as cell above. （上のセルと同じパスワードでzipファイルに保存）

%cd /content/stable-diffusion-webui
!zip -er --password={password} outputs.zip outputs 

In [ ]:
#@title Save the zip file to Google Drive. （Google Driveにzipファイルを保存）
from google.colab import drive 
drive_path = "/content/drive"
drive.mount(drive_path, force_remount=False)

!cp "/content/stable-diffusion-webui/outputs.zip" "/content/drive/MyDrive"

drive.flush_and_unmount()